### Setting up Export path for cx_Oracle. Restart the kernel after running the below cell.


In [4]:
!ls ~/instantclient.zip || curl --fail -o ~/instantclient.zip https://download.oracle.com/otn_software/linux/instantclient/193000/instantclient-basiclite-linux.x64-19.3.0.0.0dbru.zip
!mkdir -p ~/lib ; cd ~/lib ; ls libclntsh.so || unzip -j ~/instantclient.zip
!ls ~/libaio.rpm || curl --fail -o ~/libaio.rpm https://cdn-ubi.redhat.com/content/public/ubi/dist/ubi/server/7/7Server/x86_64/os/Packages/l/libaio-0.3.109-13.el7.x86_64.rpm
!mkdir -p ~/lib ; cd ~/lib ; ls libaio.so.1 || (rpm2cpio ~/libaio.rpm | cpio -idmv ) ; ln -svf lib64/libaio.so.1.0.1 libaio.so.1
!mkdir -pv $CONDA_PREFIX/etc/conda/activate.d
!echo "export LD_LIBRARY_PATH='$(realpath ~/lib)'" >$CONDA_PREFIX/etc/conda/activate.d/cxoracle.sh

/home/wsuser/instantclient.zip
libclntsh.so
/home/wsuser/libaio.rpm
libaio.so.1
'libaio.so.1' -> 'lib64/libaio.so.1.0.1'
mkdir: created directory '/opt/conda/envs/Python-3.6/etc/conda'
mkdir: created directory '/opt/conda/envs/Python-3.6/etc/conda/activate.d'


### Use IBM Supported cx_Oracle to interface with the Oracle database

### Add tables to Oracle

In [ ]:
import pandas as pd, cx_Oracle
from project_lib import Project

def connect_to_oracle():
    """
    Establish connection
    """
    project = Project.access()
    Oracle_credentials = project.get_connection(name="Oracle")
    Oracle_dsn = cx_Oracle.makedsn(host = Oracle_credentials['host'], port = Oracle_credentials['port'], service_name = Oracle_credentials['service_name'])
    Oracle_connection = cx_Oracle.connect(Oracle_credentials['username'], Oracle_credentials['password'], Oracle_dsn, encoding = 'UTF-8', nencoding = 'UTF-8')
    return Oracle_connection


def create_table(table_name,cursor,query):
    """
    Create a Table
    """
    try:
        query1 = "DROP TABLE {}".format(table_name)
        cursor.execute(query1)
    except Exception as e:
        print(e)
    cursor.execute(query)
    print("Table {} Created".format(table_name))

def insert_rows(billings,connection,sql,table_name):
    """
    Swift insertion of multiple rows  
    
    """
    try:
        # create a cursor
        with connection.cursor() as cursor:
            cursor.executemany(sql, billings)
            # commit work
            connection.commit()
            print("Values Inserted in the Table {}".format(table_name))
    except cx_Oracle.Error as error:
        import traceback
        print('Error occurred:')
        traceback.print_exc()
  
        
def fetch_values(cursor,table_name):
    """
     Fetch Values from Tables
    """
    cursor.execute('SELECT * FROM {}'.format(table_name))
    query_data = cursor.fetchall()
    col_names = [colname[0] for colname in cursor.description]
    data = pd.DataFrame(data=query_data, columns=col_names)
    return data

def create_and_batch_insert(tables,cursor,creation_dict,insertion_dict,values_dict):
    """
    Create and Batch update Tables: Driver Function
    """
    for each in tables:
        create_table(each,cursor,creation_dict[each])
        insert_rows(values_dict[each],connection,insertion_dict[each],each)

### Prepare the Python Objects to be added as batch inserts in Oracle DB

In [ ]:
import numpy as np
import pandas as pd
from utils import *
offer = pd.read_csv('/project_data/data_asset/offers.tsv', sep='\t', na_values='?')
offer = offer.replace(np.nan, '', regex=True)
offer['cpa_id']= offer['cpa_id'].astype(str)
offer['total_shipments']= offer['total_shipments'].astype(str)
values = offer.values.tolist()
#values[2]

In [ ]:
audit = pd.read_csv('/project_data/data_asset/offer_audit.tsv', sep='\t', na_values='?')
audit = audit.replace(np.nan, '', regex=True)
audit = audit.astype(str)
audit_values = audit.values.tolist()
#audit_values[0:3]

In [ ]:
discount = pd.read_csv('/project_data/data_asset/discount_policy.tsv', sep='\t', na_values='?')
discount = discount.replace(np.nan, '', regex=True)
discount = discount.astype(str)
discount_values = discount.values.tolist()

In [ ]:
shipment = pd.read_csv('/project_data/data_asset/shipment_profile.tsv', sep='\t', na_values='?')
shipment = shipment.replace(np.nan, '', regex=True)
shipment = shipment.astype(str)
shipment_values = shipment.values.tolist()
#shipment_values[0]

In [ ]:
customer = pd.read_csv('/project_data/data_asset/customers.tsv', sep='\t', na_values='?')
customer = customer.replace(np.nan, '', regex=True)
customer = customer.astype(str)
customer_values = customer.values.tolist()

In [ ]:
oppo = pd.read_csv('/project_data/data_asset/opportunities.tsv', sep='\t', na_values='?')
oppo = oppo.replace(np.nan, '', regex=True)
oppo = oppo.astype(str)
oppo_values = oppo.values.tolist()

### Construct Queries

In [ ]:
# Connect to Oracle

connection = connect_to_oracle()
cursor = connection.cursor()

offer_query = "CREATE TABLE HR.OFFER (CUSTOMER_ID varchar2(40), opportunity_id varchar2(40), offer_id varchar2(40), creation_date varchar2(40), updated_date varchar2(40), organization_id varchar2(40), offer_row_id varchar2(40), offer_status varchar2(40), offer_sub_status varchar2(40), offer_expiry_date varchar2(40), reason_for_offering varchar2(40), gsfa_customer_id varchar2(40), aag_id varchar2(40), cpa_id varchar2(40), sales_channel varchar2(40), composite_id varchar2(40), total_shipments varchar2(40), bpm_integration_id varchar2(40), effective_start_date varchar2(40), agreement_renewal_date varchar2(40), agreement_id varchar2(40), agreement_status varchar2(40), rejection_reason varchar2(40), SALES_REP_POSTN_ID varchar2(40))"
customer_query ="CREATE TABLE HR.CUSTOMERS (gsfa_customer_id varchar2(40),committed_revenue varchar2(40),created_date varchar2(40), customer_sales_stage varchar2(40), industry_code varchar2(40), loyalty_code varchar2(40), organization_id varchar2(40), physical_channel varchar2(40), reason_for_qualified_out varchar2(40), sales_channel varchar2(24), competitor varchar2(40), qualification_potential varchar2(40), ytd_revenue varchar2(40), overall_opportunity_potential varchar2(40), total_committed_revenue varchar2(40), ftb_month varchar2(40), reason_for_lead varchar2(40), lead_source_type varchar2(40))"
discount_policy_query = "CREATE TABLE HR.DISCOUNT_POLICY (billing_country varchar2(40), product_cluster varchar2(40), pid_code varchar2(40), pid_name varchar2(40), published_min_rev varchar2(40), published_max_rev varchar2(40), discount varchar2(40))"
oppo_query = "CREATE TABLE HR.OPPORTUNITIES (opportunity_id varchar2(40) ,created_date varchar2(40) ,actual_close_date varchar2(40) ,opportunity_ocr varchar2(40) ,expected_close_date varchar2(40) ,gsfa_customer_id varchar2(40) ,pipeline_stage varchar2(40) ,pipeline_last_updated_date varchar2(40) ,opportunity_type varchar2(40) ,opportunity_tpr varchar2(40) ,reason_for_lead varchar2(40) ,reason_qualified_out varchar2(40) ,primary_competitor varchar2(40) ,organization_id varchar2(40)  ,existing_committed_revenue varchar2(40) ,existing_potential_revenue varchar2(40) , new_committed_revenue varchar2(40) ,new_potential_revenue varchar2(40) ,no_of_shpmnts_order_lns varchar2(40) ,secondary_competitor varchar2(40) ,committed_revenue varchar2(40) ,lead_source varchar2(40) ,lead_source_type varchar2(40))"
shipment_query = "CREATE TABLE HR.SHIPMENT_PROFILE ( offer_id varchar2(40), created_date varchar2(40), last_updated_date varchar2(40), agreement_id varchar2(40), organization_id varchar2(40), offer_type varchar2(40), agreement_row_id varchar2(40), gsfa_customer_id varchar2(40), product_cluster varchar2(40), shipments varchar2(40), wps varchar2(40), uom varchar2(40), published_revenue varchar2(40), recommended_pid_name varchar2(40), recommended_revenue varchar2(40), pid_category varchar2(40), discount_selection varchar2(40), selected_pid_name varchar2(40), expected_revenue varchar2(40), percentage_initial_discount varchar2(40), absolute_initial_discount varchar2(40), authority_level varchar2(40), authority_name varchar2(24), avg_discount varchar2(40))"
audit_query ="CREATE TABLE HR.OFFER_AUDIT (organization_id varchar2(40), offer_id varchar2(40), created_date varchar2(40), updated_date varchar2(40), status varchar2(40), old_value varchar2(40), new_value varchar2(40), operation varchar2(40), contactrole varchar2(40))"



# Prepare Queries 

offer_sql = ('insert into HR.OFFER(CUSTOMER_ID,opportunity_id,offer_id,creation_date,updated_date,organization_id,offer_row_id,offer_status,offer_sub_status,offer_expiry_date,reason_for_offering,gsfa_customer_id,aag_id,cpa_id,sales_channel,composite_id,total_shipments,bpm_integration_id,effective_start_date,agreement_renewal_date,agreement_id,agreement_status,rejection_reason,SALES_REP_POSTN_ID) '
        'values(:CUSTOMER_ID,:opportunity_id,:offer_id,:creation_date,:updated_date,:organization_id,:offer_row_id,:offer_status,:offer_sub_status,:offer_expiry_date,:reason_for_offering,:gsfa_customer_id,:aag_id,:cpa_id,:sales_channel,:composite_id,:total_shipments,:bpm_integration_id,:effective_start_date,:agreement_renewal_date,:agreement_id,:agreement_status,:rejection_reason,:SALES_REP_POSTN_ID)')

audit_sql = ('insert into HR.OFFER_AUDIT (organization_id, offer_id, created_date, updated_date, status, old_value, new_value, operation, contactrole)'
             'values(:organization_id,:offer_id,:created_date,:updated_date,:status,:old_value,:new_value,:operation,:contactrole)')

discount_sql = ('insert into HR.DISCOUNT_POLICY(billing_country, product_cluster, pid_code, pid_name, published_min_rev, published_max_rev, discount) '
        'values(:billing_country,:product_cluster,:pid_code,:pid_name,:published_min_rev,:published_max_rev,:discount)')

ship_sql = ('insert into HR.SHIPMENT_PROFILE(offer_id, created_date, last_updated_date, agreement_id, organization_id, offer_type, agreement_row_id, gsfa_customer_id , product_cluster, shipments, wps, uom, published_revenue, recommended_pid_name, recommended_revenue, pid_category, discount_selection, selected_pid_name, expected_revenue, percentage_initial_discount, absolute_initial_discount, authority_level, authority_name , avg_discount) '
        'values(:offer_id,:created_date,:last_updated_date,:agreement_id,:organization_id,:offer_type,:agreement_row_id,:gsfa_customer_id ,:product_cluster,:shipments,:wps,:uom,:published_revenue,:recommended_pid_name,:recommended_revenue,:pid_category,:discount_selection,:selected_pid_name,:expected_revenue,:percentage_initial_discount,:absolute_initial_discount,:authority_level,:authority_name ,:avg_discount)')


oppo_sql = ('insert into HR.OPPORTUNITIES(opportunity_id ,created_date ,actual_close_date ,opportunity_ocr ,expected_close_date ,gsfa_customer_id ,pipeline_stage ,pipeline_last_updated_date ,opportunity_type ,opportunity_tpr ,reason_for_lead ,reason_qualified_out ,primary_competitor ,organization_id  ,existing_committed_revenue ,existing_potential_revenue , new_committed_revenue ,new_potential_revenue ,no_of_shpmnts_order_lns ,secondary_competitor ,committed_revenue ,lead_source ,lead_source_type) '
        'values(:opportunity_id ,:created_date ,:actual_close_date ,:opportunity_ocr ,:expected_close_date ,:gsfa_customer_id ,:pipeline_stage ,:pipeline_last_updated_date ,:opportunity_type ,:opportunity_tpr ,:reason_for_lead ,:reason_qualified_out ,:primary_competitor ,:organization_id  ,:existing_committed_revenue ,:existing_potential_revenue ,:new_committed_revenue ,:new_potential_revenue ,:no_of_shpmnts_order_lns ,:secondary_competitor ,:committed_revenue ,:lead_source ,:lead_source_type)')



customer_sql = ('insert into HR.CUSTOMERS(gsfa_customer_id,committed_revenue,created_date, customer_sales_stage, industry_code, loyalty_code, organization_id, physical_channel, reason_for_qualified_out, sales_channel , competitor, qualification_potential, ytd_revenue, overall_opportunity_potential, total_committed_revenue, ftb_month, reason_for_lead, lead_source_type) '
        'values(:gsfa_customer_id,:committed_revenue,:created_date,:customer_sales_stage,:industry_code,:loyalty_code,:organization_id,:physical_channel,:reason_for_qualified_out,:sales_channel ,:competitor,:qualification_potential,:ytd_revenue,:overall_opportunity_potential,:total_committed_revenue,:ftb_month,:reason_for_lead,:lead_source_type)')


creation_queries = [offer_query,customer_query,discount_policy_query,oppo_query,shipment_query,audit_query]

insertion_queries = [offer_sql,customer_sql,discount_sql,oppo_sql,ship_sql,audit_sql]

tables = ["HR.OFFER","HR.CUSTOMERS","HR.DISCOUNT_POLICY","HR.OPPORTUNITIES","HR.SHIPMENT_PROFILE","HR.OFFER_AUDIT"]

values_list = [values,customer_values,discount_values,oppo_values,shipment_values,audit_values]

creation_dict = dict(zip(tables,creation_queries))
insertion_dict= dict(zip(tables,insertion_queries))
values_dict = dict(zip(tables,values_list))

### Run the Following loop to create and swiftly batch-insert records to the Tables in Oracle DB

In [ ]:
create_and_batch_insert(tables,cursor,creation_dict,insertion_dict,values_dict)

In [1]:

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
Oracle_credentials = project.get_connection(name="Oracle")

import pandas as pd, cx_Oracle

Oracle_dsn = cx_Oracle.makedsn(host = Oracle_credentials['host'], port = Oracle_credentials['port'], service_name = Oracle_credentials['service_name'])
Oracle_connection = cx_Oracle.connect(Oracle_credentials['username'], Oracle_credentials['password'], Oracle_dsn, encoding = 'UTF-8', nencoding = 'UTF-8')

query = 'SELECT * FROM HR.OFFER_AUDIT'
cursor = Oracle_connection.cursor()

cursor.execute(query)
query_data = cursor.fetchall()

col_names = [colname[0] for colname in cursor.description]
data_df_1 = pd.DataFrame(data=query_data, columns=col_names)
data_df_1.head()

# You can close the database connection with the following code.
# Oracle_connection.close()
# To learn more about the cx_Oracle package, please read the documentation: https://pypi.org/project/cx-Oracle/


,ORGANIZATION_ID,OFFER_ID,CREATED_DATE,UPDATED_DATE,STATUS,OLD_VALUE,NEW_VALUE,OPERATION,CONTACTROLE
0,DHL CL,CL_O_114752510980,15 Jul 2019 08:06:32.000000,15 Jul 2019 08:06:32.000000,Status,None,Preparation,New Record,Influencer
1,DHL CL,CL_O_113694478520,13 Mar 2019 08:00:14.000000,13 Mar 2019 08:00:14.000000,Status,Offer Approved,Offer with Customer,Modify,Influencer
2,DHL CL,CL_O_115061042339,23 Aug 2019 04:26:56.000000,23 Aug 2019 04:26:56.000000,Status,None,Preparation,New Record,Influencer
3,DHL CL,CL_O_115972081300,9 Dec 2019 03:42:17.000000,9 Dec 2019 03:42:17.000000,Status,Offer with Customer,Invalidated,Modify,None
4,DHL CL,CL_O_117051190367,1 Apr 2020 08:57:09.000000,1 Apr 2020 08:57:09.000000,Status,Preparation,Approval Requested,Modify,Influencer


In [2]:
from project_lib import Project
project = Project.access()

In [3]:
import project_lib

In [5]:
project_lib.utils??

Type:        module
String form: <module 'project_lib.utils' from '/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/project_lib/utils/__init__.py'>
File:        /opt/conda/envs/Python-3.6/lib/python3.6/site-packages/project_lib/utils/__init__.py
Source:     
# -----------------------------------------------------------------------------
# Licensed Materials - Property of IBM
# 5725-Y50
# (C) Copyright IBM Corp. 2017    All Rights Reserved.
# US Government Users Restricted Rights - Use, duplication or disclosure
# restricted by GSA ADP Schedule Contract with IBM Corp.
# -----------------------------------------------------------------------------
from .logger import Logger
from .project_context import ProjectContext
from .environment import environment
from .constants import constants_dict
from .handle import ProjectHandle


In [13]:
import os
path = "/project_data/data_asset/"
os.chdir(path)
!pwd

/project_data/data_asset


In [14]:
os.makedirs("DHL_Data")

In [16]:
project.save_data('eval_mdd_test.csv',"/project_data/data_asset/DHL_Data/")

{'file_name': 'eval_mdd_test.csv',
 'message': 'File saved to project storage.',
 'asset_id': '023d5b8a-2b94-4535-9afa-b16fdc4f9817'}